### Rule Base

Imports and Initialization

In [ ]:
!pip install langdetect googletrans==4.0.0-rc1
!pip install pandas vaderSentiment
!pip install scikit-learn

In [ ]:
import pandas as pd
from langdetect import detect
from googletrans import Translator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

In [ ]:
# Initialize VADER sentiment analyzer and Google Translator
analyzer = SentimentIntensityAnalyzer()
translator = Translator()

 Data Loading

In [ ]:
import pandas as pd  # Make sure to import pandas

# Load the CSV file
file_path = 'AI in edu dataset - Sheet1.csv'  # Replace with the correct path to your file
data = pd.read_csv(file_path)

# Ensure the "Content" column exists
if 'Content' not in data.columns:
    raise ValueError('CSV file does not have a "Content" column.')

# Ensure the "Sentiment" column exists
if 'Sentiment' not in data.columns:
    raise ValueError('CSV file does not have a "Sentiment" column.')

# Ensure all values in "Content" are strings
data['Content'] = data['Content'].astype(str)

# Print Data
print(data)

# Count the occurrences of each sentiment
sentiment_counts = data['Sentiment'].value_counts()

# Print the sentiment counts
print("\nSentiment Counts:")
print(sentiment_counts)


                                               Content  Platform Sentiment
0    The education system that AI will destroy will...  facebook  Negative
1    AI tool sare very helpful but you need to use ...    tiktok  Positive
2    AI and teachers should join forces in a collab...    tiktok  Positive
3    Kids that learn how to use the AI will realize...    tiktok  Negative
4    But here’s the thing, if we embrace the AI, ho...    tiktok  Negative
..                                                 ...       ...       ...
507  AI has come and I guess we will embrace it as ...   youtube   Neutral
508  ayaw kasi mag adapt ng mga teacher. gusto kung...   youtube  Negative
509  ang masakit nasa tuktok ka na hinihila ka ng p...   youtube   Neutral
510  At first, akala ko macoconvince ako na, AI sho...   youtube   Neutral
511  Ang Ai mas makikita mo ito sa mga games kaya n...   youtube   Neutral

[512 rows x 3 columns]

Sentiment Counts:
Sentiment
Positive    266
Negative    177
Neutral      69

In [ ]:
data

,Content,Platform,Sentiment
0,The education system that AI will destroy will...,facebook,Negative
1,AI tool sare very helpful but you need to use ...,tiktok,Positive
2,AI and teachers should join forces in a collab...,tiktok,Positive
3,Kids that learn how to use the AI will realize...,tiktok,Negative
4,"But here’s the thing, if we embrace the AI, ho...",tiktok,Negative
...,...,...,...
507,AI has come and I guess we will embrace it as ...,youtube,Neutral
508,ayaw kasi mag adapt ng mga teacher. gusto kung...,youtube,Negative
509,ang masakit nasa tuktok ka na hinihila ka ng p...,youtube,Neutral
510,"At first, akala ko macoconvince ako na, AI sho...",youtube,Neutral


Language Detection and Translation

In [ ]:
def detect_language(text):
    lang = detect(text)
    return 'en' if lang == 'en' else 'tl'  # Assume non-English is Taglish

# Apply language detection to each row
data['Detected_Language'] = data['Content'].apply(detect_language)

# Separate English and Taglish content
english_content = data[data['Detected_Language'] == 'en']
taglish_content = data[data['Detected_Language'] == 'tl']

# Display results
print("WORLDWIDE Sentiments:")
print(english_content[['Content', 'Detected_Language']].head())
print("\nPH Sentiments:")
print(taglish_content[['Content', 'Detected_Language']].head())

# Count of each language
language_counts = data['Detected_Language'].value_counts()
print("\nLanguage Counts:")
print(language_counts)

WORLDWIDE Sentiments:
                                             Content Detected_Language
0  The education system that AI will destroy will...                en
1  AI tool sare very helpful but you need to use ...                en
2  AI and teachers should join forces in a collab...                en
3  Kids that learn how to use the AI will realize...                en
4  But here’s the thing, if we embrace the AI, ho...                en

PH Sentiments:
                                               Content Detected_Language
222  AI tutors are doubling student engagement in e...                tl
236  Ok sya, nagbibigay ng idea at pang research, m...                tl
237  Kung aasa kayo sa AI to do school work, kayo d...                tl
239  Nakakadismaya panahon ngayon, mga kabataan nga...                tl
240  Para sa akin, I like AI to be a mainstream thi...                tl

Language Counts:
Detected_Language
en    329
tl    183
Name: count, dtype: int64


In [ ]:
# Function to detect language and translate if not English
def detect_and_translate(text):
    # Detect language of the text
    lang = detect(text)

    # Translate to English if the detected language is not English
    if lang != 'en':
        translated = translator.translate(text, src=lang, dest='en').text
        return translated
    return text

# Loop through the first few rows and show the original and translated text
for index, row in data.head(7).iterrows():  # Adjust the number as needed
    original_text = row['Content']
    translated_text = detect_and_translate(original_text)

    print(f"Row {index + 1}") , print('=' * 700)
    print(f"Original Text: {original_text}")
    print(f"Translated Text: {translated_text}")
    print('=' * 700)



Row 1
Original Text: The education system that AI will destroy will be the system that doesn’t adapt and change to a world with AI
Translated Text: The education system that AI will destroy will be the system that doesn’t adapt and change to a world with AI
Row 2
Original Text: AI tool sare very helpful but you need to use it properly and NO they are not going to replace humans. Because you still need the human touch!
Translated Text: AI tool sare very helpful but you need to use it properly and NO they are not going to replace humans. Because you still need the human touch!
Row 3
Original Text: AI and teachers should join forces in a collaborative effort to revolutionize education. AI and teachers can empower students with the best of both.
Translated Text: AI and teachers should join forces in a collaborative effort to revolutionize education. AI and teachers can empower students with the best of both.
Row 4
Original Text: Kids that learn how to use the AI will realize what careers w

Sentiment Analysis

In [ ]:
# Function to analyze sentiment using VADER
def analyze_sentiment(text):
    sentiment_scores = analyzer.polarity_scores(text)
    compound_score = sentiment_scores['compound']  # Use compound score

    # Rule-based classification based on compound score
    if compound_score >= 0.05:
        sentiment = 'Positive'
    elif compound_score <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, sentiment_scores

# Apply language detection to each row
data['Detected_Language'] = data['Content'].apply(detect_language)

# Apply translation and sentiment analysis to each row
results = []
for index, row in data.iterrows():
    text = row['Content']
    lang = row['Detected_Language']
    translated_text = detect_and_translate(text)  # Translate if necessary
    sentiment, scores = analyze_sentiment(translated_text)

    results.append({
        'Original Content': text,
        'Translated Content': translated_text,
        'Sentiment': sentiment,
        'Compound Score': scores['compound'],
        'Positive Score': scores['pos'],
        'Neutral Score': scores['neu'],
        'Negative Score': scores['neg'],
        'Detected_Language': lang
    })

# Convert the results to a DataFrame
sentiment_df = pd.DataFrame(results)

# Save the results to a new CSV file (optional)
output_file = 'sentiment_analysis_with_translation.csv'
sentiment_df.to_csv(output_file, index=False)

# Print the first few rows of the results for inspection
print(sentiment_df.head())

                                    Original Content  \
0  The education system that AI will destroy will...   
1  AI tool sare very helpful but you need to use ...   
2  AI and teachers should join forces in a collab...   
3  Kids that learn how to use the AI will realize...   
4  But here’s the thing, if we embrace the AI, ho...   

                                  Translated Content Sentiment  \
0  The education system that AI will destroy will...  Negative   
1  AI tool sare very helpful but you need to use ...  Negative   
2  AI and teachers should join forces in a collab...  Positive   
3  Kids that learn how to use the AI will realize...  Positive   
4  But here’s the thing, if we embrace the AI, ho...  Positive   

   Compound Score  Positive Score  Neutral Score  Negative Score  \
0         -0.5423           0.000          0.851           0.149   
1         -0.4845           0.063          0.807           0.130   
2          0.7506           0.225          0.775           0.0

Sentiment Count


In [ ]:
sentiment_df

,Original Content,Translated Content,Sentiment,Compound Score,Positive Score,Neutral Score,Negative Score,Detected_Language
0,The education system that AI will destroy will...,The education system that AI will destroy will...,Negative,-0.5423,0.000,0.851,0.149,en
1,AI tool sare very helpful but you need to use ...,AI tool sare very helpful but you need to use ...,Negative,-0.4845,0.063,0.807,0.130,en
2,AI and teachers should join forces in a collab...,AI and teachers should join forces in a collab...,Positive,0.7506,0.225,0.775,0.000,en
3,Kids that learn how to use the AI will realize...,Kids that learn how to use the AI will realize...,Positive,0.2023,0.115,0.802,0.084,en
4,"But here’s the thing, if we embrace the AI, ho...","But here’s the thing, if we embrace the AI, ho...",Positive,0.4497,0.092,0.908,0.000,en
...,...,...,...,...,...,...,...,...
507,AI has come and I guess we will embrace it as ...,Ai has come and I guess we will embrace it as ...,Positive,0.3067,0.078,0.885,0.036,tl
508,ayaw kasi mag adapt ng mga teacher. gusto kung...,Do not adapt teachers.Want what they are doing...,Neutral,-0.0258,0.109,0.777,0.114,tl
509,ang masakit nasa tuktok ka na hinihila ka ng p...,The pain is at the top that your prof is pulli...,Negative,-0.7650,0.060,0.695,0.245,tl
510,"At first, akala ko macoconvince ako na, AI sho...","At first, akala ko macoconvince ako na, AI sho...",Positive,0.4215,0.107,0.850,0.043,en


In [ ]:
# Count sentiments overall
overall_sentiment_counts = sentiment_df['Sentiment'].value_counts()

# Count sentiments by language
english_sentiment_counts = sentiment_df[sentiment_df['Detected_Language'] == 'en']['Sentiment'].value_counts()
taglish_sentiment_counts = sentiment_df[sentiment_df['Detected_Language'] == 'tl']['Sentiment'].value_counts()

# Print the counts for inspection
print("Overall Sentiment Counts:")
print(overall_sentiment_counts)

print("\nWorldwide Sentiment Counts:")
print(english_sentiment_counts)

print("\nPH Sentiment Counts:")
print(taglish_sentiment_counts)


Overall Sentiment Counts:
Sentiment
Positive    285
Neutral     118
Negative    109
Name: count, dtype: int64

Worldwide Sentiment Counts:
Sentiment
Positive    214
Negative     62
Neutral      58
Name: count, dtype: int64

PH Sentiment Counts:
Sentiment
Positive    71
Neutral     60
Negative    47
Name: count, dtype: int64


### Sentiment Analysis Metrics

In [ ]:
true_labels_df = pd.read_csv('AI in edu dataset - Sheet1.csv')
predicted_labels_df = pd.read_csv('sentiment_analysis_with_translation.csv')

y_true = true_labels_df['Sentiment']
y_pred = predicted_labels_df['Sentiment']

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.2f}')

recall = recall_score(y_true, y_pred, average='weighted')
print(f'Recall: {recall:.2f}')

# Compute precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f'Precision: {precision:.2f}')

# Compute F1 score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 Score: {f1:.2f}')

# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)


Accuracy: 0.56
Recall: 0.56
Precision: 0.59
F1 Score: 0.56
Confusion Matrix:
[[ 68  41  68]
 [ 15  27  27]
 [ 26  50 190]]
